In [1]:
from importlib import reload

import datasets
import numpy as np

import src.models.components.feature_extractor_dinov2
import src.models.components.spatiotemporal_encoder

In [4]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [7]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(src_tensor, sampling_rate=25)


print("input_values:", batched_feature['input_values'].shape)
print("attention_mask:", batched_feature['attention_mask'])

input_values: torch.Size([6, 3, 768])
attention_mask: tensor([[1., 1., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.]], dtype=torch.float64)


In [9]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder


model = SpatiotemporalEncoder()
encoded = model(batched_feature)

encoded.shape

torch.Size([6, 3, 768])

In [12]:
from src.models.components.sign_language_net import SignLanguageNet

model = SignLanguageNet()
out = model.generate(batched_feature)

out.shape

AttributeError: 